# Imports

In [1]:
import pickle
import json
from __future__ import division
import warnings
warnings.filterwarnings('ignore')
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from numpy.random import randint
from numpy.random import rand
import multiprocessing as mp
import random
import math
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from tqdm.contrib.concurrent import process_map
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras import layers
from keras.layers import Input,Dense, Dropout, Flatten, Reshape, GlobalAveragePooling1D, BatchNormalization,LSTM,Bidirectional
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D,UpSampling1D, Conv1DTranspose,GaussianNoise 
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import Callback
from keras.regularizers import l2 , l1, l1_l2
import tensorflow.keras.backend as kb
print('keras version ', keras.__version__)

from keras.utils.vis_utils import plot_model

keras version  2.9.0


In [2]:
sys.path.append("../src")
from convertor import convert
from preprocessing import get_target, preprocessing

In [3]:
from imblearn.under_sampling import RandomUnderSampler

# Functions

In [4]:
def df_to_fasta(df, path):
    lines = []
    df.apply(lambda row: lines.append(f">{row['tag']}\n{row['data']}\n"),axis=1)
    with open(path,'w') as file:
        file.write(''.join(lines))

In [5]:
def fasta_to_df(path):
    with open(path, 'r') as file:
        text = file.read()
    lines = [line for line in text.split('\n') if len(line) > 0]
    s = ''
    tags = []
    data = []
    for l in lines:
        if(l[0]=='>'):
            tags.append(l)        
            data.append(s)
            s = ''
        else:
            s += l    
    data.append(s)
    df = pd.DataFrame(
            {
                'tag': tags,
                'data': data[1:]
            })
    df['tag'] = df['tag'].apply(lambda x: x[1:])    
    return df

In [6]:
class DotDict(dict):
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

In [7]:
def confusion_matrix(validations, predictions, LABELS):            
    freq = metrics.confusion_matrix(validations, predictions)        
    sensitivity =  (np.divide(freq.transpose(),
                             np.sum(freq, axis=1)).transpose() * 100).round(2)    
    annotation = []
    for i in range(freq.shape[0]):
        row = []
        for j in range(freq.shape[1]):
            row.append(f'{freq[i][j]}\n {sensitivity[i][j]}%')            
        annotation.append(row)
    annotation = np.array(annotation)
    sns.heatmap(sensitivity, cmap="YlOrBr",
                vmin=0, vmax=100,
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=annotation,
               fmt=':<')    
    plt.title("Confusion Matrix")
    plt.ylabel("Actually Label")
    plt.xlabel("Predicted Label")
    plt.show()            

def report(pred, df, ref):
    number_of_hit = len(df['hit seq'].unique())
    print(f'number of unique hit: {number_of_hit}')
    total = len(df[df['hit seq'].isin(ref['data'])]['hit seq'].unique()) 
    print(f'total mir: {total}')    
    selected = df[pd.Series(pred).apply(lambda x: True if x==1 else False)]
    found_seq = selected[selected['hit seq'].isin(ref['data'])]['hit seq'].unique()
    found = len(found_seq)
    print(f'found mir : {found}')
    print(f'fount to all : {round(found / total * 100, 2)}%')
    fp = len(selected[~selected['hit seq'].isin(ref['data'])]['hit seq'].unique())
    print(f'fp: {fp}')

# Load all plant dataset

In [8]:
all_plant = [
    "O.sativa",
    "A.thaliana",
    "C.sinensis",
    "Z.mays",
    "T.aestivum",
    "G.max",
    "G.raimondii",
    "M.truncatula",
    "S.bicolor",
    #"H.vulgare"
]

In [9]:
data = {}

In [10]:
def get_load(plant, mu=None, std=None, under_sampling=False):
    posX = pd.read_csv(f"../experiment/{plant}/positive/Result/result_level1_filter.csv")
    posX = convert(posX)    
    posY = np.ones(posX.shape[0], dtype='float32')
    posY = np_utils.to_categorical(posY, 2)
    print(f'Print positive shape: {posY.shape[0]}')
    
    negX = pd.read_csv(f"../experiment/{plant}/negative/Result/result_level1_filter.csv")
    negX = convert(negX)    
    negY = np.zeros(negX.shape[0], dtype='float32')
    negY = np_utils.to_categorical(negY, 2)        
    print(f'Print negative shape: {negY.shape[0]}')
    Y = np.concatenate([posY, negY])            
    X = pd.concat([posX, negX]).reset_index(drop=True)
    [feature, mu, std] = preprocessing(X, mu, std)    
    data[plant] = {}    
    if(under_sampling):
        rus = RandomUnderSampler(random_state=0)  
        feature.columns = [str(i) for i in feature.columns]
        feature, Y = rus.fit_resample(feature , Y)        
        display(feature)
        Y = np_utils.to_categorical(Y, 2)         
    data[plant]['df'] = X
    data[plant]['feature'] = feature
    data[plant]['target'] = Y
    print(f'total data: {feature.shape[0]}')
    return [X, feature, Y, mu, std]

def load(plant, mu=None, std=None, under_sampling=True):
    global data
    data[plant] = {}        
    [X, feature, Y, mu, std] = get_load(plant, mu, std, under_sampling)
    data[plant]['df'] = X
    data[plant]['feature'] = feature
    data[plant]['target'] = Y
    return [mu, std]

In [11]:
with open('../data/classifier/mu.pickle', 'rb') as handle:
    mu = pickle.load(handle)
with open('../data/classifier/std.pickle', 'rb') as handle:
    std = pickle.load(handle)

## O.sativa

In [12]:
load(plant="O.sativa", mu=mu, std=std);

Print positive shape: 1419
Print negative shape: 11556


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,2.338811,-0.810982,1.423757,-0.548806,1.404141,-0.019986,-0.890041,-2.130301,2.037410,2.863688,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,2.338811,-1.429180,1.955559,-1.057230,0.552293,-0.180070,-1.314551,-0.754888,0.355337,1.667516,...,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2,-1.076685,2.975481,-1.682977,2.946608,-0.459276,-0.388179,2.718290,0.932424,-0.991195,-1.458039,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,-1.076685,2.511833,-1.995267,2.311078,-1.257884,-0.628304,2.435284,0.982164,-0.914149,-1.390325,...,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
4,2.338811,-1.274630,2.621520,-0.548806,1.191179,-0.019986,-1.173048,-1.455618,1.569123,2.938665,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2833,1.655712,-0.038234,0.152297,0.340936,0.978217,-0.035994,-0.111774,-0.360240,-0.070966,-0.113936,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2834,1.655712,-0.038234,0.080348,0.340936,0.978217,-0.035994,-0.111774,-0.026024,0.268605,0.170043,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2835,1.655712,-0.038234,0.062461,0.340936,0.978217,-0.035994,-0.182525,0.015043,0.242551,0.199451,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2836,1.655712,-0.038234,0.011580,0.340936,0.978217,-0.035994,-0.111774,-0.269806,-0.184245,-0.208669,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


total data: 2838


## A.thaliana

In [13]:
load(plant="A.thaliana", mu=mu, std=std);

Print positive shape: 847
Print negative shape: 5464


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,-1.076685,0.425414,-0.836620,0.086724,-0.193074,0.588333,0.312736,0.135247,-0.316439,-0.529240,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1,-1.076685,-0.579158,-0.572669,-0.548806,-0.991681,-0.564271,-0.465532,1.291711,-1.661964,-1.361975,...,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,-1.076685,-0.501883,-1.055289,-1.184336,-0.459276,-0.372170,-1.031545,0.879848,-0.968292,-0.360596,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
3,-0.393586,-0.579158,-0.517960,-1.883419,0.126369,-0.804397,-1.526806,1.271006,-1.293338,-0.361632,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
4,-0.393586,0.193590,-1.084870,-0.675912,0.126369,-0.852422,-0.394780,1.239941,-1.235548,-0.988686,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1689,-0.393586,0.039041,-0.498765,-0.358147,0.658774,-0.404187,0.029729,0.412392,-0.492839,-0.536429,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
1690,-0.393586,0.039041,-0.521761,-0.358147,0.658774,-0.404187,0.029729,0.417981,-0.417084,-0.475772,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0
1691,-0.393586,0.270865,-0.113859,-0.040382,0.658774,-0.244103,0.100481,0.008727,0.300747,0.053091,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0
1692,-0.393586,0.270865,-0.060304,1.167124,0.871736,-0.244103,-0.111774,0.479449,0.221737,0.130848,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


total data: 1694


## C.sinensis

In [14]:
load(plant="C.sinensis", mu=mu, std=std);

Print positive shape: 540
Print negative shape: 914


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,-0.393586,-1.120081,-0.676711,-0.993677,-0.885200,-0.276120,-1.173048,0.924894,-1.140702,-0.460620,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,-0.393586,-1.506455,0.134305,-0.675912,0.392572,0.044048,-1.456054,0.398812,-0.977094,0.029076,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,-0.393586,-1.351905,0.110012,-0.675912,-0.352795,-0.884439,-1.385303,0.737294,-0.587263,0.493410,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,-0.393586,-1.274630,0.045354,-0.675912,-0.139833,0.044048,-1.173048,0.674939,-0.732163,0.051864,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,0.289513,-1.351905,-0.105446,-0.803018,0.179610,0.060056,-1.314551,0.793809,-0.968394,-0.111621,...,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,1.655712,-0.965531,0.095341,-0.993677,0.499053,-0.180070,-0.748538,0.479493,-0.302432,0.141712,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1076,1.655712,-0.965531,0.069531,-0.993677,0.499053,-0.180070,-0.748538,0.499380,-0.383274,0.057203,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1077,1.655712,-0.965531,0.163031,-0.993677,0.499053,-0.180070,-0.748538,0.450227,-0.285592,0.159316,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
1078,1.655712,-0.965531,0.131377,-0.993677,0.499053,-0.180070,-1.102296,0.109202,-0.680454,0.045080,...,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


total data: 1080


## Z.mays

In [15]:
load(plant="Z.mays", mu=mu, std=std);

Print positive shape: 625
Print negative shape: 1384


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,0.289513,0.116315,-1.390501,-0.231041,-0.512517,-0.084019,0.100481,0.788367,-1.134774,-1.072384,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.393586,0.039041,-0.740486,0.213830,0.126369,-0.180070,-0.182525,0.494515,-0.588515,-0.511354,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
2,-0.393586,1.661810,-1.447796,0.213830,-0.352795,-0.804397,1.374010,0.652629,-0.318776,-0.888560,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.393586,-0.501883,-0.436830,-0.040382,0.392572,-0.324145,-0.394780,0.629511,-0.569516,-0.377352,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
4,-0.393586,0.966338,-1.564515,0.213830,-0.352795,-0.852422,1.020252,0.951148,-0.388706,-0.832045,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,-0.393586,0.039041,-0.033351,-0.040382,0.871736,-0.052003,0.241984,0.131243,0.269841,-0.056469,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1246,0.289513,0.039041,0.043723,0.086724,0.445812,-0.084019,0.241984,0.076870,0.374866,0.022578,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1247,0.289513,0.039041,-0.017569,0.086724,0.445812,-0.084019,0.241984,-0.102059,0.179599,-0.124391,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
1248,0.289513,0.039041,-0.081063,0.086724,0.445812,-0.084019,0.241984,0.156292,0.325243,-0.014771,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0


total data: 1250


## T.aestivum

In [16]:
load(plant="T.aestivum", mu=mu, std=std);

Print positive shape: 421
Print negative shape: 10363


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,0.289513,0.657239,0.637946,0.404489,1.137939,-0.164061,0.807997,-0.919413,1.428983,0.413161,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,-0.393586,-0.115509,-1.035473,-0.993677,-0.618998,-0.804397,-0.111774,1.048029,-1.113491,-0.985785,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
2,-0.393586,-0.038234,-1.355984,0.213830,-1.630567,-0.676330,-0.182525,1.146981,-0.974623,-0.841589,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0.289513,0.734513,0.183874,0.404489,0.658774,-0.340154,0.737245,-0.921936,1.057954,0.212365,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
4,-0.393586,-0.270059,-1.075640,-0.040382,-1.364365,-0.852422,0.029729,1.399722,-1.834724,-1.610876,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,1.655712,0.889063,0.342350,0.595148,0.765255,-0.404187,0.524991,0.011827,1.496659,0.646809,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
838,1.655712,0.889063,0.409511,0.595148,0.765255,-0.404187,0.524991,0.011827,1.496659,0.646809,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
839,1.655712,0.889063,0.321530,0.595148,0.765255,-0.404187,0.524991,-0.060540,1.350943,0.545259,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
840,1.655712,0.889063,0.315485,0.595148,0.765255,-0.404187,0.241984,-0.209400,0.665667,0.241452,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


total data: 842


## G.max

In [17]:
load(plant="G.max", mu=mu, std=std);

Print positive shape: 1115
Print negative shape: 5008


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,1.655712,-0.579158,1.217594,-0.167488,1.191179,-0.356162,-0.607035,-0.364891,0.422494,0.741419,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.289513,0.193590,-0.158397,0.086724,0.179610,0.028039,0.171233,0.226136,0.264179,-0.019024,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
2,-0.393586,-0.347333,-0.248598,-0.358147,-0.352795,-0.660321,-0.536283,0.955610,-0.725966,-0.439752,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
3,0.289513,-0.965531,0.736746,-1.057230,0.658774,-0.820405,-0.890041,0.296070,-0.048539,0.554857,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
4,-0.393586,0.193590,-0.809834,-0.993677,-1.364365,-0.772380,-0.111774,0.765621,-0.766606,-0.695689,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2225,0.289513,-0.579158,0.460345,0.404489,0.658774,-0.035994,-0.536283,-0.290047,-0.172699,0.094121,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2226,-1.759785,-1.506455,0.310504,-2.773161,0.818496,0.300182,-1.456054,0.303143,-1.023087,-0.037494,...,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2227,-1.759785,-1.506455,0.308578,-2.773161,-1.098162,0.108081,-1.385303,0.625344,-0.950938,-0.013484,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
2228,-1.759785,-1.506455,0.329768,-2.773161,0.286091,0.156107,-1.385303,0.549479,-0.900340,0.057040,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0


total data: 2230


## G.raimondii

In [18]:
load(plant="G.raimondii", mu=mu, std=std);

Print positive shape: 822
Print negative shape: 10576


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,1.655712,-0.424608,-0.162144,0.595148,0.286091,0.780434,0.241984,-0.470016,0.299960,-0.033800,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,-0.393586,-0.038234,0.539000,0.531595,0.658774,-0.292128,0.312736,-0.508467,0.732032,0.244426,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
2,-0.393586,-0.656432,0.628405,0.531595,-0.139833,0.764425,-0.182525,-1.092283,0.521694,0.438200,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
3,-0.393586,0.425414,1.260944,0.213830,1.137939,0.604341,0.241984,-1.888879,1.379504,0.778727,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,1.655712,-0.810982,-0.451633,-0.421700,-0.618998,-0.628304,-0.536283,1.046359,-1.091055,-0.792043,...,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1639,-0.393586,-0.965531,0.979166,-0.993677,1.137939,1.740937,-1.031545,-1.343683,-0.123042,0.633441,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1640,-0.393586,-0.965531,0.975415,-0.993677,1.137939,1.740937,-1.031545,-1.340271,-0.125797,0.630201,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1641,-0.393586,-0.965531,1.126507,-0.993677,1.137939,1.740937,-1.031545,-1.454894,-0.033257,0.739031,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1642,-0.393586,-0.501883,0.799026,-0.993677,1.404141,-0.148053,-0.607035,-0.263837,0.555321,0.872962,...,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


total data: 1644


## M.truncatula

In [19]:
load(plant="M.truncatula", mu=mu, std=std);

Print positive shape: 971
Print negative shape: 5613


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,-0.393586,0.116315,-1.980143,0.213830,-2.642137,-0.756372,-0.182525,1.208496,-1.730115,-1.487756,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-0.393586,-0.656432,-0.614578,-0.993677,-0.885200,1.885012,-0.819290,-0.323214,-1.065946,-0.615064,...,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.289513,-0.270059,-0.325278,0.404489,-0.778719,1.756945,-0.324029,-0.433028,-0.545645,-0.397083,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
3,0.972613,-1.738279,0.609165,-0.612359,0.499053,2.781482,-1.456054,-0.824502,-0.736436,0.377409,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.972613,-1.506455,0.164865,-1.184336,-0.193074,-0.884439,-0.960793,0.841101,-0.930817,-0.370702,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1937,1.655712,-0.965531,1.984297,-0.167488,1.191179,0.316190,-1.031545,-0.857823,0.766837,1.679963,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1938,1.655712,-0.965531,2.045955,-0.167488,1.191179,0.316190,-0.960793,-1.005148,0.698670,1.487550,...,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1939,-0.393586,-0.270059,0.888449,0.531595,1.137939,-0.580279,-0.324029,0.052568,0.448911,0.494060,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1940,-0.393586,-0.270059,0.728098,0.531595,1.137939,-0.580279,-0.324029,0.117115,0.323056,0.381291,...,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


total data: 1942


## S.bicolor

In [20]:
load(plant="S.bicolor", mu=mu, std=std);

Print positive shape: 528
Print negative shape: 3841


,hit len,flanking GC content,flanking MFEI,hit GC content,hit complementarity percentage,num of linking residues,boi GC content,boi delta G,boi AMFE,boi MFEI,...,connectivity hit start-2,connectivity hit end-2,connectivity hit start-1,connectivity hit end-1,connectivity hit start,connectivity hit end,connectivity hit start+1,connectivity hit end+1,connectivity hit start+2,connectivity hit end+2
0,-0.393586,-1.351905,1.021719,-0.993677,0.871736,0.732408,-1.102296,-0.428360,-0.191092,0.639148,...,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
1,-0.393586,1.198162,-0.910347,0.213830,-0.618998,-0.788388,1.161755,1.050144,-0.385377,-0.870448,...,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
2,-1.076685,0.657239,-1.103597,0.404489,-0.193074,-0.324145,0.524991,0.628598,-0.723144,-0.900183,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,1.655712,0.889063,0.092721,0.595148,0.765255,-0.868430,1.161755,0.171239,2.511579,0.860041,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0
4,-0.393586,0.657239,-1.154844,0.531595,-1.151403,-0.772380,1.020252,1.088628,-0.993804,-1.205349,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1051,-0.393586,-0.656432,3.130143,-0.993677,1.404141,2.493331,-0.677787,-5.305283,1.489761,1.903855,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1052,-0.393586,-0.656432,3.159102,-0.993677,1.404141,2.493331,-0.677787,-5.337751,1.508454,1.922868,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1053,-0.393586,-0.656432,3.347333,-0.358147,1.404141,2.493331,-0.677787,-5.551383,1.631444,2.047961,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1054,-0.393586,-0.656432,3.130143,-0.358147,1.404141,2.493331,-0.677787,-5.305283,1.489761,1.903855,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


total data: 1056


# Correlation

In [37]:
data_list = []
for plan in all_plant:
    data_list.append(data[plan]['feature'])
_data = pd.concat(data_list)

In [38]:
correlation_matrix = _data.corr()

In [44]:
highly_correlated = set()

threshold = 1
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            col_i = correlation_matrix.columns[i]
            col_j = correlation_matrix.columns[j]
            highly_correlated.add((col_i, col_j))

print("Pairs of highly correlated features:")
for pair in highly_correlated:
    print(pair)

Pairs of highly correlated features:
